In [0]:
%tensorflow_version 2.x

In [0]:
import json
import tensorflow as tf
import numpy as np

from model.input_fn import input_fn
from model.model_fn import model_fn

tf.keras.backend.clear_session()

# 파라미터 불러오기

*   ./params 디렉토리 밑에 있는 JSON 파일을 로드.
*   dataset_params.json, model_params.json, training_params.json



In [0]:
with open('./params/dataset_params.json') as f:
    data_params = json.load(f)

with open('./params/model_params.json') as f:
    model_params = json.load(f)

with open('./params/training_params.json') as f:
    training_params = json.load(f)

# 데이터 불러오기

*   구글 드라이브에 업로드 된 학습 데이터를 로드하여, dataset 딕셔너리 초기화.
*   nsmc/train/sentences.txt, nsmc/train/labels.txt 등 총 6개 파일.

```
dataset = {
    'train': {
        'sentences': [List],
        'labels': [List]
    },
    'dev': {
        'sentences': [List],
        'labels': [List]

    },
    'test': {
        'sentences': [List],
        'labels': [List]
    }
}
```





In [0]:
dataset_fields = ['sentences', 'labels']
dataset_splits = ['train', 'dev', 'test']

dataset = dict()
for split in dataset_splits[:3]:
    dataset[split] = dict()
    for field in dataset_fields:
        with open(f'./drive/My Drive/colab/nsmc/{split}/{field}.txt') as f:
            lines = f.readlines()
            dataset[split][field] = tuple([line.strip() for line in lines])


# 데이터 정수 인코딩


*   input_fn에서 각 단어를 정수 형태로 인코딩하여, 문장을 벡터로 변환 후 동일한 길이로 padding 처리.
*   vocab의 크기와 문장 벡터의 차원은 data_params['vocab_size']과 data_params['max_sentence_length']을 이용.
*   긍정(1)과 부정(0) label은 바로 numpy 형태로 변환



In [0]:
training_sentences = input_fn(dataset['train']['sentences'], data_params)
training_labels = np.asarray([int(label) for label in dataset['train']['labels']])
dev_sentences = input_fn(dataset['dev']['sentences'], data_params)
dev_labels = np.asarray([int(label) for label in dataset['dev']['labels']])

# 모델 생성

In [0]:
model = model_fn(data_params, model_params)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

# 모델 학습

In [0]:
batch_size = training_params['batch_size']
epochs = training_params['epochs']

with tf.device('/GPU:0'):
    history = model.fit(training_sentences, training_labels, 
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(dev_sentences, dev_labels),
                        shuffle=True)

Train on 136630 samples, validate on 29278 samples
Epoch 1/30
136630/136630 [==============================] - 43s 316us/sample - loss: 0.7103 - accuracy: 0.5135 - val_loss: 0.7054 - val_accuracy: 0.5593
Epoch 2/30
136630/136630 [==============================] - 41s 303us/sample - loss: 0.6300 - accuracy: 0.7216 - val_loss: 0.8450 - val_accuracy: 0.5645
Epoch 3/30
136630/136630 [==============================] - 42s 306us/sample - loss: 0.5685 - accuracy: 0.7856 - val_loss: 0.8573 - val_accuracy: 0.5560
Epoch 4/30
136630/136630 [==============================] - 42s 305us/sample - loss: 0.5489 - accuracy: 0.8019 - val_loss: 0.9305 - val_accuracy: 0.5571
Epoch 5/30
136630/136630 [==============================] - 41s 303us/sample - loss: 0.5457 - accuracy: 0.8058 - val_loss: 1.0686 - val_accuracy: 0.5487
Epoch 6/30
136630/136630 [==============================] - 41s 303us/sample - loss: 0.5446 - accuracy: 0.8096 - val_loss: 0.9358 - val_accuracy: 0.5454
Epoch 7/30
136630/136630 [=====

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
model.save_weights('weight.h5')

In [0]:
test_sentences = input_fn(dataset['test']['sentences'], data_params)
test_labels = np.asarray([int(label) for label in dataset['test']['labels']])

loss, acc = model.evaluate(test_sentences, test_labels)
print(loss, acc)